In [ ]:
!pip list | grep -E "dotenv"
!mkdir -p ../data/fed ../data/finance ../data/stock

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
print(f"Current Working Directory: {os.getcwd()}")

FRED_api_key = os.getenv("FRED_API_KEY")



In [ ]:
import yfinance as 
import pandas as pd

tickers = ["AAPL", "TSLA", "MSFT", "AMZN", "GOOGL"]
data = {}
path_stock = "../data/stock"
#path = "/Users/payamoghtanem/bootcamp/capstone-SignalSigma/data"
for ticker in tickers:
    df = yf.download(ticker, start="2000-01-01", end="2025-04-22", interval="1d")
    df['Ticker'] = ticker
    data[ticker] = df
    df.to_csv(f"{path_stock}/{ticker}_stock.csv")

In [ ]:
AAPL_df = pd.read_csv(f"{path_stock}/AAPL_stock.csv")


In [ ]:
AMZN_df = pd.read_csv(f"{path_stock}/AMZN_stock.csv")


In [ ]:
GOOGL_df = pd.read_csv(f"{path_stock}/GOOGL_stock.csv")


In [ ]:
MSFT_df = pd.read_csv(f"{path_stock}/MSFT_stock.csv")


In [ ]:
TSLA_df = pd.read_csv(f"{path_stock}/TSLA_stock.csv")


In [ ]:
import requests
import pandas as pd


tickers = ["AAPL", "TSLA", "MSFT", "AMZN", "GOOGL"]
FINANCE_api_key = os.getenv("FINANCE_API_KEY")


# Function to fetch data from FMP API and create a DataFrame with time series index
def fetch_fmp_data(ticker, endpoint):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{ticker}?limit=5&apikey={FINANCE_api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list) and all(isinstance(item, dict) for item in data):
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date']) # Convert 'date' column to datetime
            df.set_index('date', inplace=True) # Set 'date' as index
            return df
        else:
            print(f"Unexpected data format for {ticker}: {data}")
    else:
        print(f"API request failed for {ticker} with status code: {response.status_code}")
        print(f"Error message: {response.text}")
    return None

# Fetch financial statement data
income_statements = {}
balance_sheets = {}
cash_flow_statements = {}

for ticker in tickers:
    income_statements[ticker] = fetch_fmp_data(ticker, "income-statement") # Income Statement
    balance_sheets[ticker] = fetch_fmp_data(ticker, "balance-sheet-statement") # Balance Sheet
    cash_flow_statements[ticker] = fetch_fmp_data(ticker, "cash-flow-statement") # Cash Flow Statement

# Fetch key metrics
key_metrics = {}
for ticker in tickers:
    key_metrics[ticker] = fetch_fmp_data(ticker, "key-metrics") # Key Metrics

# Example: Accessing Apple's income statement
apple_income_statement = income_statements["AAPL"] 

# To see the output, run the code.
# apple_income_statement.head()  # Display the first 5 rows
path_finance = "../data/finance"
# Save DataFrames to CSV files
for ticker in tickers:
    income_statements[ticker].to_csv(f"{path_finance}/{ticker}_income_statement.csv")
    balance_sheets[ticker].to_csv(f"{path_finance}/{ticker}_balance_sheet.csv")
    cash_flow_statements[ticker].to_csv(f"{path_finance}/{ticker}_cash_flow_statement.csv")
    key_metrics[ticker].to_csv(f"{path_finance}/{ticker}_key_metrics.csv")

In [ ]:
from fredapi import Fred # Import the Fred class from the fredapi library to interact with the FRED API
import pandas as pd # Import the pandas library for data manipulation and analysis
import holidays # Import the holidays library to access holiday information

def get_economic_indicators(api_key, path):
    """
    Fetches economic indicators from FRED, converts to daily frequency,
    filters for data after 2000, adds day of week and holiday columns,
    filters for working days only, and returns a pandas DataFrame.
    
    Args:
        api_key (str): Your FRED API key.
        path (str): The path to save the CSV file.
    
    Returns:
        pandas.DataFrame: A DataFrame containing the economic indicators.
        https://fred.stlouisfed.org
    """
    fred = Fred(api_key=api_key) # Create a Fred object using your API key
    
    # Economic indicators and their FRED series IDs
    # These indicators provide insights into various aspects of the economy:
    indicators = {
        "cpi": "CPIAUCSL",  # Consumer Price Index (CPI): Measures inflation and purchasing power.
        "fed_rate": "FEDFUNDS",  # Federal Funds Rate: Target rate for overnight lending between banks.
        "vix": "VIXCLS",  # VIX Volatility Index: Measures market volatility and risk.
        "oil": "DCOILWTICO",  # Crude Oil Prices: Reflects energy costs and global economic conditions.
        "gdp": "GDP",  # Gross Domestic Product (GDP): Measures overall economic activity and growth.
        "nonfarm_payrolls": "PAYEMS",  # Nonfarm Payrolls: Number of jobs added or lost in the economy.
        "treasury_yield": "DGS10",  # 10-Year Treasury Yield: Reflects long-term interest rates.
        "industrial_production": "INDPRO",  # Industrial Production Index: Measures output of industrial sectors.
        "retail_sales": "RSXFS",  # Retail Sales: Reflects consumer spending and economic health.
        "pmi": "MANEMP",  # Manufacturing PMI: Indicates business conditions in the manufacturing sector.
        "consumer_confidence": "UMCSENT"  # Consumer Confidence Index: Reflects consumer sentiment and spending outlook.
    }

    df = pd.DataFrame()  # Create an empty DataFrame to store the economic indicators

    # Loop through the indicators and fetch data from FRED
    for name, series_id in indicators.items():
        data = fred.get_series(series_id) # Fetch the data for the current indicator using its series ID
        data = data.resample('D').ffill()  # Resample to daily frequency and forward fill missing values
        df[name] = data  # Add the data as a column to the DataFrame using the indicator name as the column name

    # Filter data after 2000-01-01 to focus on more recent economic trends
    df = df[df.index >= pd.to_datetime('2000-01-01')]

    # Add a column for the day of the week to analyze potential day-of-week effects
    df['day_of_week'] = df.index.day_name()

    # Add a column to indicate whether a date is a US holiday
    us_holidays = holidays.US() # Create a US holidays object
    df['is_holiday'] = [date in us_holidays for date in df.index] # Check if each date in the index is a holiday
    
    # Add a column to indicate whether a date is a working day (not a holiday or weekend)
    df['is_working_day'] = ~df['is_holiday'] & (df['day_of_week'] != 'Saturday') & (df['day_of_week'] != 'Sunday')
    
    # Filter the DataFrame to include only working days for analysis purposes
    df = df[df['is_working_day']]  
    

    # Save the DataFrame to a CSV file for future use
    path_fed = "../data/fed"

    df.to_csv(f"{path_fed}/combined_economic_indicators.csv", index=True) 
    
    # Return the DataFrame containing the economic indicators
    return df

# Example usage:
 # Replace with your actual API key

path_fed = "../data/fed"
economic_indicators_df = get_economic_indicators(FRED_api_key, path_fed) # Call the function to get the economic indicators DataFrame

# Print the first few rows of the DataFrame to inspect the data
print(economic_indicators_df.head())